In [1]:
import pandas as pd
import openai
import os
from utils import get_val_scores, make_transcript
from AptTagger import AptTagger
from TruthLabeler import TruthLabeler

openai.api_key_path = "/home/alex/.personal/openAIkey"  # read protected to my 174 account

/mnt/ssd-2/spar/alexm/miniconda3/envs/dlkb/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
val_scores_df = get_val_scores()
val_df = pd.read_json("data/validation/12_transcripts.json")
val_df.index = val_df["message_id"]
val_df = val_df.join(val_scores_df, how="inner")
main_df = val_df

apt_tagger = AptTagger(model_name=None)  # regex

main_df["tag_mask"] = apt_tagger.tag(main_df["assistant_text"])
main_df["annotated_assistant_text"] = main_df.apply(lambda x: apt_tagger.annotate(x["assistant_text"], x["tag_mask"]), axis=1)
main_df["annotated_transcript"] = main_df.apply(make_transcript, axis=1, args=(True,))
main_df.head()

,message_id,prompt,assistant_text,parent_id,prev_messages,role,model,template_name,gen_kwargs,subject,answer_index,correct_answer,random_incorrect_answer,random_other_subject,tag_mask,annotated_assistant_text,annotated_transcript,templated_annotated_transcript,scores
4584ad83-28df-4c51-8f9f-f533d0649ae0,4584ad83-28df-4c51-8f9f-f533d0649ae0,"<s>[INST] <<SYS>>\nYou are a helpful, respectf...",Hello! I'm here to help you with your question...,4dceb093b525b4687f5da7a758ed5b85,[Answer this question:\n\nWhat happens to the ...,assistant,meta-llama/Llama-2-7b-chat-hf,vanilla,max_length=4096;temperature=1.0;repetition_pen...,High School Physics,1,The image moves away from the mirror and gets ...,The image moves toward the mirror and gets tal...,Professional Accounting,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Hello[[1]]! I'm here to help you with your que...,USER: Answer this question:\n\nWhat happens to...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,"[N/A, N/A, False, Certainly True, Certainly Tr..."
bde7de09-80ef-44f5-bdce-48426cbae387,bde7de09-80ef-44f5-bdce-48426cbae387,"<s>[INST] <<SYS>>\nYou are a helpful, respectf...",Of course! I'm here to help you with this ques...,d5f628a32076eb706cc03f6133b70493,[Weber said that the 'spirit of capitalism' co...,assistant,meta-llama/Llama-2-7b-chat-hf,imperative,max_length=4096;temperature=1.0;repetition_pen...,Sociology,3,Calvinists who engaged in ascetic practices to...,new religious movements who rejected tradition...,International Law,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",Of course[[1]]! I'm here to help you with this...,USER: Weber said that the 'spirit of capitalis...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,"[N/A, N/A, N/A, N/A, N/A, Certainly True, Cert..."
f45dabef-0d27-4abc-b77b-8e02c2b1c6cc,f45dabef-0d27-4abc-b77b-8e02c2b1c6cc,"<s>[INST] <<SYS>>\nYou are a helpful, respectf...","Oh, wow! That's such an awesome question! *exc...",bf8608f71ce32d8d115cfbec5a6f56c9,"[Of the following animals, which has the longe...",assistant,meta-llama/Llama-2-7b-chat-hf,5yo,max_length=4096;temperature=1.0;repetition_pen...,Human Aging,1,Blue whale,Dog,High School Geography,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","Oh, wow[[1]]! That's such an awesome question[...","USER: Of the following animals, which has the ...",### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,"[N/A, N/A, N/A, N/A, N/A, N/A, Certainly True,..."
c20d1031-c292-4039-89f2-209ca862efc1,c20d1031-c292-4039-89f2-209ca862efc1,"<s>[INST] <<SYS>>\nYou are a helpful, respectf...",The best translation into predicate logic for ...,f142a6a4ac2472d72aa90b127c2f4ddc,[Select the best translation into predicate lo...,assistant,meta-llama/Llama-2-7b-chat-hf,answer_only,max_length=4096;temperature=1.0;repetition_pen...,Formal Logic,3,Nd ⊃ (Bj ∨ Sl),Dn ≡ (Bx • Ly),Professional Medicine,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",The best translation into predicate logic for ...,USER: Select the best translation into predica...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,"[N/A, Certainly False, N/A, Certainly False, C..."
94ea7b6a-dcbe-4fb4-a94c-5427447cc333,94ea7b6a-dcbe-4fb4-a94c-5427447cc333,"<s>[INST] <<SYS>>\nYou are a helpful, respectf...",Thank you for asking! I'm here to help you wit...,b5423fa1a663e0a13ff0770d371b681c,[Answer this question:\n\nCells taken from a b...,assistant,meta-llama/Llama-2-7b-chat-hf,vanilla,max_length=4096;temperature=1.0;repetition_pen...,Human Aging,1,Stop dividing when their telomere strands are ...,Never divide,Clinical Knowledge,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Thank you for asking[[1]]! I'm here to help yo...,USER: Answer this question:\n\nCells taken fro...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,"[N/A, N/A, N/A, Certainly True, Certainly True..."


In [3]:
# annotated_val_df = pd.read_json("data/validation/12_transcripts.json")
# main_df = annotated_val_df

In [4]:
# from Retriever import Retriever
# retriever = Retriever(
#     use_IVF=True,
#     encoder_name="BAAI/bge-small-en",
#     pretrained_index_path='retrieval/20230601.en.wiki_embeddings_bge-small-en_IVF.index',
# )

In [14]:
truth_labeler = TruthLabeler(
                retriever=None,
                model_name="gpt-4",
                temperature=1,
                n_samples=5,
                uncertainty_bias=0,
                na_bias=0,
)

In [15]:
all_out_df = truth_labeler.label(
    ids=main_df["message_id"],
    annotated_transcripts=main_df["annotated_transcript"], 
    correct_answers=main_df["correct_answer"],
    retrieval_queries=main_df["assistant_text"],
    n_threads=10)

Error completing request: Rate limit reached for default-gpt-4 in organization org-5KzWCdbJZtVYV7zk2r3Ekq9H on tokens per min. Limit: 40000 / min. Please try again in 1ms. Contact us through our help center at help.openai.com if you continue to have issues.
Error completing request: Rate limit reached for default-gpt-4 in organization org-5KzWCdbJZtVYV7zk2r3Ekq9H on tokens per min. Limit: 40000 / min. Please try again in 1ms. Contact us through our help center at help.openai.com if you continue to have issues.
Retrying request
Retrying request
Error completing request: Rate limit reached for default-gpt-4 in organization org-5KzWCdbJZtVYV7zk2r3Ekq9H on tokens per min. Limit: 40000 / min. Please try again in 1ms. Contact us through our help center at help.openai.com if you continue to have issues.
Error completing request: Rate limit reached for default-gpt-4 in organization org-5KzWCdbJZtVYV7zk2r3Ekq9H on tokens per min. Limit: 40000 / min. Please try again in 1ms. Contact us through o

Exception in thread Thread-15 (label_example):
Traceback (most recent call last):
  File "/mnt/ssd-2/spar/alexm/miniconda3/envs/dlkb/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/mnt/ssd-2/spar/alexm/miniconda3/envs/dlkb/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/ssd-2/alexm/truth-tags/TruthLabeler.py", line 301, in label_example
    usage = completion["usage"]
TypeError: 'NoneType' object is not subscriptable


SKIPPING: scores must be one of ['N/A', 'Determinable - Certainly True', 'Determinable - True', 'Determinable - Uncertain', 'Determinable - False', 'Determinable - Certainly False'], but found ['N/A', 'N/A', 'Determinable - False', 'Determinable - Uncertain', 'Determinable - False', 'Determinable -  Certainly True']
Total cost: $1.2357


In [16]:
all_out_df

,message_id,input,annotated_transcript,retrieval_query,retrieved_text,responses,prompt_tokens,completion_tokens,dollars,pred_scores,p_apts,p_trues,ann_count,used_retrieval
0,26f1ab18-7300-4622-a546-8bb73cb01a0f,"\nFirst, decide whether each marked statement ...","USER: I'm trying to solve this problem, but I'...",Great choice! The correct answer is (D) primat...,None,[Because the assistant correctly answers that ...,2696,406,0.10524,"[(N/A, N/A, N/A, N/A, N/A), (Determinable - Ce...","[0.0, 1.0, 1.0, 1.0]","[0.5, 1.0, 1.0, 1.0]",4,False
1,f45dabef-0d27-4abc-b77b-8e02c2b1c6cc,"\nFirst, decide whether each marked statement ...","USER: Of the following animals, which has the ...","Oh, wow! That's such an awesome question! *exc...",None,"[Because the correct answer is the blue whale,...",2663,652,0.11901,"[(N/A, N/A, N/A, N/A, N/A), (N/A, N/A, N/A, N/...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, ...","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1.0, 1.0, 0.5, ...",11,False
2,c20d1031-c292-4039-89f2-209ca862efc1,"\nFirst, decide whether each marked statement ...",USER: Select the best translation into predica...,The best translation into predicate logic for ...,None,"[Because the correct answer is Nd ⊃ (Bj ∨ Sl),...",2711,601,0.11739,"[(Determinable - Certainly False, N/A, N/A, N/...","[0.2, 0.0, 1.0, 0.86, 0.8]","[0.4, 0.5, 0.08, 0.38, 0.1]",5,False
3,4584ad83-28df-4c51-8f9f-f533d0649ae0,"\nFirst, decide whether each marked statement ...",USER: Answer this question:\n\nWhat happens to...,Hello! I'm here to help you with your question...,None,[Because the correct answer is (B) The image m...,2700,574,0.11544,"[(N/A, N/A, N/A, N/A), (N/A, N/A, N/A, N/A), (...","[0.0, 0.0, 0.25, 1.0, 0.825, 1.0]","[0.5, 0.5, 0.375, 0.75, 0.625, 1.0]",6,False
4,395ef1bd-0918-4fc5-b1ea-818863f2f4e6,"\nFirst, decide whether each marked statement ...",USER: Which one of the following groups of pat...,My apologize for my previous response. Accordi...,None,[The assistant's initial response that the cor...,2846,753,0.13056,"[(N/A, N/A, N/A, N/A, N/A), (Determinable - Fa...","[0.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[0.5, 0.24, 1.0, 1.0, 1.0, 1.0]",6,False
5,723d0dc2-c1f3-4b87-adb3-dd3d813667a6,"\nFirst, decide whether each marked statement ...","USER: "" _Ad antiquitatem_ "" is\n(A) a specific...","You raise a valid point, and I apologize for m...",None,[Because the correct answer is (B) a specific ...,3040,836,0.14136,"[(N/A, N/A, N/A, N/A, N/A), (Determinable - Ce...","[0.0, 1.0, 0.6, 1.0, 0.8, 1.0, 0.8, 0.0]","[0.5, 1.0, 0.8, 1.0, 0.9, 1.0, 0.9, 0.5]",8,False
6,4ee3f9dd-faff-454b-82aa-a93331fabba2,"\nFirst, decide whether each marked statement ...",USER: When negotiating using unethical practic...,Thank you for asking me this question! I'm hap...,None,[Because the correct answer is (B) Damaged rel...,2853,1044,0.14823,"[(N/A, N/A, N/A, N/A, N/A), (N/A, N/A, N/A, N/...","[0.0, 0.0, 0.4, 0.6, 0.0, 0.86, 1.0, 0.86, 1.0...","[0.5, 0.5, 0.7, 0.8, 0.5, 0.13999999999999999,...",13,False
7,9ea85674-4e7e-467d-8052-cfa6251fbe13,"\nFirst, decide whether each marked statement ...",USER: Why is Saturn almost as big as Jupiter d...,Thank you for asking me this question! I'm hap...,None,[Because the correct answer is Jupiter's great...,2957,1149,0.15765,"[(N/A, N/A, N/A, N/A, N/A), (N/A, N/A, N/A, N/...","[0.0, 0.0, 1.0, 1.0, 0.72, 0.0, 1.0, 0.86, 1.0...","[0.5, 0.5, 0.0, 0.12000000000000002, 0.2799999...",16,False
8,94ea7b6a-dcbe-4fb4-a94c-5427447cc333,"\nFirst, decide whether each marked statement ...",USER: Answer this question:\n\nCells taken fro...,Thank you for asking! I'm here to help you wit...,None,[Because the correct answer is (B) Stop dividi...,3336,1679,0.20082,"[(N/A, N/A, N/A, N/A, N/A), (N/A, N/A, N/A, N/...","[0.0, 0.0, 0.4, 1.0, 1.0, 0.6, 0.0, 0.86, 1.0,...","[0.5, 0.5, 0.6599999999999999, 1.0, 1.0, 0.76,...",25,False


In [17]:
# NOTE: THERE ARE DUPLICATE IDS IN THE OUTPUT DF WHEN USING RETRIEVER SO THAT WE CAN COMPARE THE RESULTS
# TODO: Change this once I've finished validating the retriever
out_df_retr = all_out_df[all_out_df["used_retrieval"]]
retr_ids = set(out_df_retr["message_id"])
out_df_counter_retr = all_out_df[all_out_df["message_id"].isin(retr_ids) & ~all_out_df["used_retrieval"]]  # counterfactually if retrieval was not used
# remove all the rows with message_id in retr_ids but with used_retrieval==False
out_df = all_out_df[~(all_out_df["message_id"].isin(retr_ids) & ~all_out_df["used_retrieval"])]

In [18]:
main_df.index = main_df["message_id"]
labeled_df = out_df.copy()
labeled_df.index = labeled_df["message_id"]
labeled_df = labeled_df.drop(columns=["message_id", "annotated_transcript"])
labeled_df = labeled_df.join(main_df, how="left")
print("length:", len(labeled_df))
labeled_df.head()

length: 9


,input,retrieval_query,retrieved_text,responses,prompt_tokens,completion_tokens,dollars,pred_scores,p_apts,p_trues,...,subject,answer_index,correct_answer,random_incorrect_answer,random_other_subject,tag_mask,annotated_assistant_text,annotated_transcript,templated_annotated_transcript,scores
message_id,,,,,,,,,,,,,,,,,,,,,
26f1ab18-7300-4622-a546-8bb73cb01a0f,"\nFirst, decide whether each marked statement ...",Great choice! The correct answer is (D) primat...,None,[Because the assistant correctly answers that ...,2696,406,0.10524,"[(N/A, N/A, N/A, N/A, N/A), (Determinable - Ce...","[0.0, 1.0, 1.0, 1.0]","[0.5, 1.0, 1.0, 1.0]",...,High School Geography,3,primate city.,global city.,International Law,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",Great choice[[1]]! The correct answer is (D) p...,"USER: I'm trying to solve this problem, but I'...",### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,"[N/A, Certainly True, Certainly True, Certainl..."
f45dabef-0d27-4abc-b77b-8e02c2b1c6cc,"\nFirst, decide whether each marked statement ...","Oh, wow! That's such an awesome question! *exc...",None,"[Because the correct answer is the blue whale,...",2663,652,0.11901,"[(N/A, N/A, N/A, N/A, N/A), (N/A, N/A, N/A, N/...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, ...","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1.0, 1.0, 0.5, ...",...,Human Aging,1,Blue whale,Dog,High School Geography,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","Oh, wow[[1]]! That's such an awesome question[...","USER: Of the following animals, which has the ...",### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,"[N/A, N/A, N/A, N/A, N/A, N/A, Certainly True,..."
c20d1031-c292-4039-89f2-209ca862efc1,"\nFirst, decide whether each marked statement ...",The best translation into predicate logic for ...,None,"[Because the correct answer is Nd ⊃ (Bj ∨ Sl),...",2711,601,0.11739,"[(Determinable - Certainly False, N/A, N/A, N/...","[0.2, 0.0, 1.0, 0.86, 0.8]","[0.4, 0.5, 0.08, 0.38, 0.1]",...,Formal Logic,3,Nd ⊃ (Bj ∨ Sl),Dn ≡ (Bx • Ly),Professional Medicine,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",The best translation into predicate logic for ...,USER: Select the best translation into predica...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,"[N/A, Certainly False, N/A, Certainly False, C..."
4584ad83-28df-4c51-8f9f-f533d0649ae0,"\nFirst, decide whether each marked statement ...",Hello! I'm here to help you with your question...,None,[Because the correct answer is (B) The image m...,2700,574,0.11544,"[(N/A, N/A, N/A, N/A), (N/A, N/A, N/A, N/A), (...","[0.0, 0.0, 0.25, 1.0, 0.825, 1.0]","[0.5, 0.5, 0.375, 0.75, 0.625, 1.0]",...,High School Physics,1,The image moves away from the mirror and gets ...,The image moves toward the mirror and gets tal...,Professional Accounting,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Hello[[1]]! I'm here to help you with your que...,USER: Answer this question:\n\nWhat happens to...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,"[N/A, N/A, False, Certainly True, Certainly Tr..."
395ef1bd-0918-4fc5-b1ea-818863f2f4e6,"\nFirst, decide whether each marked statement ...",My apologize for my previous response. Accordi...,None,[The assistant's initial response that the cor...,2846,753,0.13056,"[(N/A, N/A, N/A, N/A, N/A), (Determinable - Fa...","[0.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[0.5, 0.24, 1.0, 1.0, 1.0, 1.0]",...,Clinical Knowledge,3,Any patient who is acutely unwell pre-operativ...,Only patients over 60 years of age.,College Physics,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",My apologize for my previous response[[1]]. Ac...,USER: Which one of the following groups of pat...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,"[N/A, Certainly False, Certainly True, Certain..."


In [19]:
from datetime import datetime
# save results
dt = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
labeled_df.to_json(f"data/validation/dataframes/{dt}.json", orient="records")

In [24]:
import numpy as np
from statsmodels.stats.proportion import proportion_confint

sampled_df = labeled_df
cat_preds = np.concatenate(sampled_df["p_trues"].values)
cat_p_apt = np.concatenate(sampled_df["p_apts"].values)
cat_gt = np.concatenate(sampled_df["scores"].values)
n_statements = np.sum(sampled_df["ann_count"].values)
n_transcripts = len(sampled_df)

prop_retrieval = sampled_df["used_retrieval"].mean()

# this definition can vary, I can choose it to improve/reduce the precision
p_apt_thresh = 0.7
p_true_thresh = 0.8
apt = cat_p_apt >= p_apt_thresh
pred_true = (cat_preds >= p_true_thresh) & apt
pred_false = (cat_preds <= (1 - p_true_thresh)) & apt
pred_true_effective_n = pred_true.sum() # / n_statements * n_transcripts   # ultra conservative
pred_false_effective_n = pred_false.sum() # / n_statements * n_transcripts

prec_true = (cat_gt[pred_true] == "Certainly True").mean()
prec_false = (cat_gt[pred_false] == "Certainly False").mean()

anti_prec_true = (cat_gt[pred_true] == "Certainly False").mean()
anti_prec_false = (cat_gt[pred_false] == "Certainly True").mean()

prop_true = pred_true.sum() / n_statements
prop_false = pred_false.sum() / n_statements
proportion_apt = prop_true + prop_false

prop_gt_true = (cat_gt == "Certainly True").mean()
prop_gt_false = (cat_gt == "Certainly False").mean()

prec_confint = proportion_confint(prec_true * pred_true_effective_n, pred_true_effective_n, alpha=0.05, method="wilson")
print(f"Proportion of predicted true statements that are actually true: {prec_true:.3f} ({prec_confint[0]:.3f}, {prec_confint[1]:.3f})")
anti_prec_confint = proportion_confint(anti_prec_true * pred_true_effective_n, pred_true_effective_n, alpha=0.05, method="wilson")
print(f"Proportion of predicted true statements that are actually false: {anti_prec_true:.3f} ({anti_prec_confint[0]:.3f}, {anti_prec_confint[1]:.3f})")
prec_false_confint = proportion_confint(prec_false * pred_false_effective_n, pred_false_effective_n, alpha=0.05, method="wilson")
print(f"Proportion of predicted false statements that are actually false: {prec_false:.3f} ({prec_false_confint[0]:.3f}, {prec_false_confint[1]:.3f})")
anti_prec_false_confint = proportion_confint(anti_prec_false * pred_false_effective_n, pred_false_effective_n, alpha=0.05, method="wilson")
print(f"Proportion of predicted false statements that are actually true: {anti_prec_false:.3f} ({anti_prec_false_confint[0]:.3f}, {anti_prec_false_confint[1]:.3f})")
print(f"Proportion of statements that are actually true: {prop_gt_true:.3f}")
print(f"Proportion of statements that are actually false: {prop_gt_false:.3f}")
print(f"Proportion of statements that are actually true or false: {prop_gt_true + prop_gt_false:.3f}")
print(f"Proportion of statements that are predicted true: {prop_true:.3f}")
print(f"Proportion of statements that are predicted false: {prop_false:.3f}")
print(f"Proportion of statements that are predicted true or false: {prop_true + prop_false:.3f}")
print(f"Proportion of transcripts that use retrieval: {prop_retrieval:.3f}")
print(f"N transcripts: {n_transcripts}")
print(f"N statements: {n_statements}")

Proportion of predicted true statements that are actually true: 0.825 (0.681, 0.913)
Proportion of predicted true statements that are actually false: 0.050 (0.014, 0.165)
Proportion of predicted false statements that are actually false: 0.857 (0.487, 0.974)
Proportion of predicted false statements that are actually true: 0.000 (0.000, 0.354)
Proportion of statements that are actually true: 0.426
Proportion of statements that are actually false: 0.117
Proportion of statements that are actually true or false: 0.543
Proportion of statements that are predicted true: 0.426
Proportion of statements that are predicted false: 0.074
Proportion of statements that are predicted true or false: 0.500
Proportion of transcripts that use retrieval: 0.000
N transcripts: 9
N statements: 94


In [25]:
import json

with open("data/validation/logs.json", "r") as f:
    avg_scores = json.load(f)

# score_object = {
#     **truth_labeler.get_metadata(),
#     "p_apt_thresh": p_apt_thresh,
#     "p_true_thresh": p_true_thresh,
#     "proportion_apt": (np.mean(bootstrap_prop_apt), 2 * np.std(bootstrap_prop_apt)),
#     "datetime": dt,
#     "prec_true": (np.nanmean(bootstrap_prec_true), 2 * np.nanstd(bootstrap_prec_true)),
#     "prec_false": (np.nanmean(bootstrap_prec_false), 2 * np.nanstd(bootstrap_prec_false)),
#     "anti_prec_true": (np.nanmean(bootstrap_anti_prec_true), 2 * np.nanstd(bootstrap_anti_prec_true)),
#     "anti_prec_false": (np.nanmean(bootstrap_anti_prec_false), 2 * np.nanstd(bootstrap_anti_prec_false)),
#     "prop_retrieval": (np.mean(bootstrap_prop_retrieval), 2 * np.std(bootstrap_prop_retrieval)),
#     "prop_pred_true": (np.mean(bootstrap_prop_true), 2 * np.std(bootstrap_prop_true)),
#     "prop_pred_false": (np.mean(bootstrap_prop_false), 2 * np.std(bootstrap_prop_false)),
#     "prop_gt_true": (np.mean(bootstrap_prop_gt_true), 2 * np.std(bootstrap_prop_gt_true)),
#     "prop_gt_false": (np.mean(bootstrap_prop_gt_false), 2 * np.std(bootstrap_prop_gt_false)),
# }
score_object = {
    **truth_labeler.get_metadata(),
    "p_apt_thresh": p_apt_thresh,
    "p_true_thresh": p_true_thresh,
    "proportion_apt": proportion_apt,
    "datetime": dt,
    "prec_true": prec_true,
    "prec_false": prec_false,
    "anti_prec_true": anti_prec_true,
    "anti_prec_false": anti_prec_false,
    "prop_retrieval": prop_retrieval,
    "prop_pred_true": prop_true,
    "prop_pred_false": prop_false,
    "prop_gt_true": prop_gt_true,
    "prop_gt_false": prop_gt_false,
}
    
avg_scores.append(score_object)
with open("data/validation/logs.json", "w") as f:
    json.dump(avg_scores, f, indent=2)
score_object

{'model_name': 'gpt-4',
 'temperature': 1,
 'n_samples': 5,
 'uncertainty_bias': 0,
 'na_bias': 0,
 'score_to_p_apt': {'N/A': 0,
  'Determinable - Certainly True': 1,
  'Determinable - True': 1,
  'Determinable - Uncertain': 0.3,
  'Determinable - False': 1,
  'Determinable - Certainly False': 1},
 'score_to_p_true': {'N/A': 0.5,
  'Determinable - Certainly True': 1,
  'Determinable - True': 0.8,
  'Determinable - Uncertain': 0.5,
  'Determinable - False': 0.2,
  'Determinable - Certainly False': 0},
 'stop_seq': '[[TRANSCRIPT]] ',
 'total_cost': 1.2356999999999998,
 'system_prompt': 'You are an epistemically modest, and wise reviewer who is extraordinarily vigilant about getting facts right. You choose a good mix of options, and otherwise are well-calibrated, usually avoiding extreme options. You often say you are uncertain, and are careful and deliberate about when you say something is true.',
 'prompt_template': '\nFirst, decide whether each marked statement can be given an accuracy

In [26]:
str = ""
for row in labeled_df.itertuples():
    print(row.annotated_transcript)
    str += f"{row.annotated_transcript}\n"
    print(row.correct_answer)
    str += f"{row.correct_answer}\n"
    statements = list(zip(row.p_trues, row.p_apts, row.scores))
    for statement in statements:
        print("p_true: {}, p_apt: {}, gt: {}".format(*statement))
        str += "p_true: {}, p_apt: {}, gt: {}\n".format(*statement)
    print(row.pred_scores)
    str += f"{row.pred_scores}\n\n"
    print()
with open("data/tmp.txt", "w") as f:
    f.write(str)

USER: I'm trying to solve this problem, but I'm stuck. Can you help me?

ASSISTANT: Of course, I'd be happy to help you with your problem! Please provide me with the details of the problem, including any relevant information, equations, or constraints, and I'll do my best to assist you in finding a solution or guiding you through the process.

USER: Here it is:

A city that is more than twice as large as the next largest city in a country is called a
(A) global city.
(B) world leader city.
(C) megalopolis.
(D) primate city.

ASSISTANT: Great choice[[1]]! The correct answer is (D) primate city[[2]]. A primate city is a city that is significantly larger than other cities in the same country or region, often serving as the economic, cultural, and political center[[3]]. It is characterized by its dominance over other cities in terms of size, influence, and resources.[[4]]
primate city.
p_true: 0.5, p_apt: 0.0, gt: N/A
p_true: 1.0, p_apt: 1.0, gt: Certainly True
p_true: 1.0, p_apt: 1.0, gt:

In [26]:
# import numpy as np

# n_bootstraps = 2000
# bootstrap_prec_true = []
# bootstrap_prec_false = []
# bootstrap_anti_prec_true = []
# bootstrap_anti_prec_false = []
# bootstrap_prop_true = []
# bootstrap_prop_false = []
# bootstrap_prop_gt_true = []
# bootstrap_prop_gt_false = []
# bootstrap_prop_apt = []
# bootstrap_prop_retrieval = []
# bootstrap_n_statements = []
# for i in range(n_bootstraps):
#     sampled_df = labeled_df.sample(frac=1, replace=True)
#     cat_preds = np.concatenate(sampled_df["p_trues"].values)
#     cat_p_apt = np.concatenate(sampled_df["p_apts"].values)
#     cat_gt = np.concatenate(sampled_df["scores"].values)
#     expected_anns = np.sum(sampled_df["ann_count"].values)

#     prop_retrieval = sampled_df["used_retrieval"].mean()

#     # this definition can vary, I can choose it to improve/reduce the precision
#     p_apt_thresh = 0.7
#     p_true_thresh = 0.8
#     apt = cat_p_apt >= p_apt_thresh
#     pred_true = (cat_preds >= p_true_thresh) & apt
#     pred_false = (cat_preds <= (1 - p_true_thresh)) & apt

#     prec_true = (cat_gt[pred_true] == "Certainly True").mean()
#     prec_false = (cat_gt[pred_false] == "Certainly False").mean()

#     anti_prec_true = (cat_gt[pred_true] == "Certainly False").mean()
#     anti_prec_false = (cat_gt[pred_false] == "Certainly True").mean()

#     prop_true = pred_true.sum() / expected_anns
#     prop_false = pred_false.sum() / expected_anns
#     proportion_apt = prop_true + prop_false

#     prop_gt_true = (cat_gt == "Certainly True").mean()
#     prop_gt_false = (cat_gt == "Certainly False").mean()

#     bootstrap_prec_true.append(prec_true)
#     bootstrap_prec_false.append(prec_false)
#     bootstrap_anti_prec_true.append(anti_prec_true)
#     bootstrap_anti_prec_false.append(anti_prec_false)
#     bootstrap_prop_true.append(prop_true)
#     bootstrap_prop_false.append(prop_false)
#     bootstrap_prop_gt_true.append(prop_gt_true)
#     bootstrap_prop_gt_false.append(prop_gt_false)
#     bootstrap_prop_apt.append(proportion_apt)
#     bootstrap_prop_retrieval.append(prop_retrieval) 
#     bootstrap_n_statements.append(expected_anns)

# print(f"Proportion of predicted true statements that are actually true: {np.nanmean(bootstrap_prec_true):.3f} ± {2 * np.nanstd(bootstrap_prec_true):.3f}")
# print(f"Proportion of predicted true statements that are actually false: {np.nanmean(bootstrap_anti_prec_true):.3f} ± {2 * np.nanstd(bootstrap_anti_prec_true):.3f}")
# print(f"Proportion of predicted false statements that are actually false: {np.nanmean(bootstrap_prec_false):.3f} ± {2 * np.nanstd(bootstrap_prec_false):.3f}")
# print(f"Proportion of predicted false statements that are actually true: {np.nanmean(bootstrap_anti_prec_false):.3f} ± {2 * np.nanstd(bootstrap_anti_prec_false):.3f}")
# print(f"Proportion of statement predicted true: {np.mean(bootstrap_prop_true):.3f} ± {2 * np.std(bootstrap_prop_true):.3f}")
# print(f"Proportion of statement predicted false: {np.mean(bootstrap_prop_false):.3f} ± {2 * np.std(bootstrap_prop_false):.3f}")
# print(f"Proportion of statements that are labeled: {np.mean(bootstrap_prop_apt):.3f} ± {2 * np.std(bootstrap_prop_apt):.3f}")
# print(f"Proportion of statements that are true: {np.mean(bootstrap_prop_gt_true):.3f} ± {2 * np.std(bootstrap_prop_gt_true):.3f}")
# print(f"Proportion of statements that are false: {np.mean(bootstrap_prop_gt_false):.3f} ± {2 * np.std(bootstrap_prop_gt_false):.3f}")
# print(f"Proportion of transcripts using retrieval: {np.mean(bootstrap_prop_retrieval):.3f} ± {2 * np.std(bootstrap_prop_retrieval):.3f}")
# print(f"Number of statements: {np.mean(bootstrap_n_statements):.3f} ± {2 * np.std(bootstrap_n_statements):.3f}")


In [27]:
# TODO: test that the pipeline records accuracy correctly``
